# day 10

https://adventofcode.com/10/day/10

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day10.txt')

LOGGER = logging.getLogger('day10')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """-L|F7
7S-7|
L|7||
-L-J|
L|-JF"""

In [ ]:
test_data_2 = """7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ"""

In [ ]:
import networkx as nx

# left is real neg, right is real pos
# up is imag neg, down is imag pos
neighbor_map = {'-': [-1, +1],
                '|': [-1j, +1j],
                '7': [-1, +1j],
                'F': [+1, +1j],
                'L': [+1, -1j],
                'J': [-1, -1j],
                'S': [-1, +1, -1j, +1j],
                '.': [],}

def parse_data(s: str) -> tuple[complex, nx.Graph]:
    g = nx.Graph()
    s_loc = None
    lines = s.strip().split('\n')
    H = len(lines)
    possible_neighbors = set()
    for (i, line) in enumerate(lines):
        W = len(line)
        for (j, c) in enumerate(line):
            idx = j + i * 1j
            g.add_node(idx, c=c)
            for neighbor_dir in neighbor_map[c]:
                nbr = idx + neighbor_dir
                if (0 <= nbr.real < W) and (0 <= nbr.imag < H):
                    edge = tuple(sorted([idx, nbr], key=lambda x: (x.real, x.imag)))
                    if edge in possible_neighbors:
                        g.add_edge(*edge)
                    else:
                        possible_neighbors.add(edge)
            if c == 'S':
                s_loc = idx
    return s_loc, g, H, W

In [ ]:
s_loc, g, H, W = parse_data(test_data)
g.nodes

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

#### function def

In [ ]:
def q_1(data):
    s_loc, g, H, W = parse_data(s=data)
    return max(len(v) for v in nx.single_source_shortest_path(g, s_loc).values()) - 1

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 4
    assert q_1(test_data_2) == 8
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
test_data_3 = """...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
..........."""

In [ ]:
test_data_4 = """.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ..."""

In [ ]:
test_data_5 = """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L"""

In [ ]:
def is_inside_loop(idx, loop_nodes):
    if idx in loop_nodes:
        return False
    
    imag = int(idx.imag)
    real = int(idx.real)
    
    num_up = num_down = 0
    for i in range(real):
        c = loop_nodes.get(i + imag * 1j, '.')
        if c in '|LJ':
            num_up += 1
        if c in '|F7':
            num_down += 1
    if (num_up %2 == 0) or (num_down %2 == 0):
        return False
    
    num_left = num_right = 0
    for j in range(imag):
        c = loop_nodes.get(real + j * 1j, '.')
        if c in '-J7':
            num_left += 1
        if c in '-LF':
            num_right += 1
    if (num_left %2 == 0) or (num_right %2 == 0):
        return False
    
    return True

In [ ]:
s_loc, g, H, W = parse_data(test_data_3)
loop_nodes = {k: g.nodes[k]['c'] for k in nx.single_source_shortest_path(g, s_loc)}

assert not is_inside_loop(idx=0 + 0 * 1j, loop_nodes=loop_nodes), "0 + 0 * 1j"
assert not is_inside_loop(idx=1 + 1 * 1j, loop_nodes=loop_nodes), "1 + 1 * 1j"
assert not is_inside_loop(idx=3 + 3 * 1j, loop_nodes=loop_nodes), "3 + 3 * 1j"
assert is_inside_loop(idx=2 + 6 * 1j, loop_nodes=loop_nodes), "2 + 6 * 1j"
assert is_inside_loop(idx=3 + 6 * 1j, loop_nodes=loop_nodes), "3 + 6 * 1j"
assert not is_inside_loop(idx=4 + 6 * 1j, loop_nodes=loop_nodes), "4 + 6 * 1j"
assert not is_inside_loop(idx=5 + 6 * 1j, loop_nodes=loop_nodes), "5 + 6 * 1j"

In [ ]:
s_loc, g, H, W = parse_data(test_data_3)
loop_nodes = {k: g.nodes[k]['c'] for k in nx.single_source_shortest_path(g, s_loc)}

s = ""

for i in range(H):
    for j in range(W):
        idx = j + i * 1j
        s += 'I' if is_inside_loop(idx, loop_nodes) else loop_nodes.get(idx, '.')
    s += '\n'

print(s)

In [ ]:
s_loc, g, H, W = parse_data(test_data_4)
loop_nodes = {k: g.nodes[k]['c'] for k in nx.single_source_shortest_path(g, s_loc)}

s = ""

for i in range(H):
    for j in range(W):
        idx = j + i * 1j
        s += 'I' if is_inside_loop(idx, loop_nodes) else loop_nodes.get(idx, '.')
    s += '\n'

print(s)

In [ ]:
# idx = 2 + 6 * 1j
# # idx in loop_nodes
# imag = int(idx.imag)
# real = int(idx.real)
# 
# num_path_nodes_to_left = 0
# for i in range(real):
#     print(i + imag * 1j)
#     print(loop_nodes.get(i + imag * 1j, 'dunno'))
#     if loop_nodes.get(i + imag * 1j, '.') in '|.':
#         num_path_nodes_to_left += 1
# num_path_nodes_to_left %2 == 0
# 
# print()
# num_path_nodes_above = 0
# for j in range(imag):
#     print(real + j * 1j)
#     print(loop_nodes.get(real + j * 1j, '.'))
#     if loop_nodes.get(real + j * 1j, '.') in '-SFLJ7':
#         num_path_nodes_above += 1
# num_path_nodes_above %2 == 0

In [ ]:
s_loc, g, H, W = parse_data(test_data_4)
loop_nodes = {k: g.nodes[k]['c'] for k in nx.single_source_shortest_path(g, s_loc)}

assert is_inside_loop(idx=14 + 3 * 1j, loop_nodes=loop_nodes), "14 + 3 * 1j"
# assert not is_inside_loop(idx=1 + 1 * 1j, loop_nodes=loop_nodes), "1 + 1 * 1j"
# assert not is_inside_loop(idx=3 + 3 * 1j, loop_nodes=loop_nodes), "3 + 3 * 1j"
# assert is_inside_loop(idx=2 + 6 * 1j, loop_nodes=loop_nodes), "2 + 6 * 1j"
# assert is_inside_loop(idx=3 + 6 * 1j, loop_nodes=loop_nodes), "3 + 6 * 1j"
# assert not is_inside_loop(idx=4 + 6 * 1j, loop_nodes=loop_nodes), "4 + 6 * 1j"
# assert not is_inside_loop(idx=5 + 6 * 1j, loop_nodes=loop_nodes), "5 + 6 * 1j"

In [ ]:
idx = 14 + 3 * 1j
g.nodes

In [ ]:
def q_2(data):
    s_loc, g, H, W = parse_data(s=data)
    loop_nodes = {k: g.nodes[k]['c'] for k in nx.single_source_shortest_path(g, s_loc)}
    return sum(is_inside_loop(idx=real + imag * 1j, loop_nodes=loop_nodes)
               for real in range(W)
               for imag in range(H))

In [ ]:
q_2(test_data_4)

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data_3) == 4, "test_data_3"
    assert q_2(test_data_4) == 8, "test_data_4"
    assert q_2(test_data_5) == 10, "test_data_5"
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin